In [1]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import scipy
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as ag
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [2]:
dfr = hd.read_raw_data("777")

Time for read_raw_data                 : 6.12 seconds


In [83]:
dft = hd.read_txn_data("X77")

Time for read_txn_data                 : 35.12 seconds


In [89]:
dfs = hd.read_txn_data("777")

Time for read_txn_data                 : 2.94 seconds


In [4]:
dfa = hd.read_analysis_data("777")

Time for read_analysis_data            : 0.54 seconds


## e and es component correlation

In [117]:
data = dfs

# unsmoothed entropy
counts = ag._entropy_base_values(data, "tag_spend")
e = ag._entropy_scores(counts)

# calc smooth entropy and components
# es = a + b, where a is pos count component and b is zero count component
es = ag._entropy_scores(counts, smooth=True)
row_totals = counts.sum(1)
num_unique = len(counts.columns)
zero_counts = (counts == 0).sum(1)
const = (1 / (row_totals + num_unique)) * np.log2(1 / (row_totals + num_unique))
b = zero_counts * -const
smooth_probs_pos_counts = (counts > 0) * (counts + 1).div(row_totals + num_unique, axis=0)
a = scipy.special.entr(smooth_probs_pos_counts).sum(1) / np.log(2)
pd.testing.assert_series_equal(a + b, es)

e.corr(a), e.corr(b), a.corr(b)



(0.890642746287733, -0.7482122369481141, -0.9507259196683358)